<a href="https://colab.research.google.com/github/ingridyrs/Rede-de-Artistas-Relacionados-A-Milton-Nascimento-no-Spotify/blob/insper/Milton_Nascimento_Rede_de_artistas_relacionados_Spotify_%3E_Gephi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Baseado [nesse notebook](https://gist.github.com/Kautenja/71f139eee58099b77e91a0d775e42b47#file-visualizing-spotify-related-artists-with-gephi-ipynb) de @Kautenja. 

# Spotify API + Spotipy

In [ ]:
!pip install spotipy

     |████████████████████████████████| 63 kB 905 kB/s 
     |████████████████████████████████| 138 kB 58.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
# spotify credentials for the application:
# "Related Artist Network Visualizer"
#Código Original: client_id = 'COLOQUE A SUA AQUI'
#Código Original: client_secret = 'COLOQUE A SUA AQUI'
client_id = '10b23ac5208a441994d915e5d5a818f1'
client_secret = '4568c89108764e7ba943fe4f6b8d97a8'
# create a credential manager and api layer
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

## Artista Base

O artista escolhido é o [Milton Nascimento](https://open.spotify.com/artist/1jb6b6gCRJfS9ffzBWF17d?autoplay=true).

In [ ]:
seed_artist = 'spotify:artist:3Bnq7jiU506HcPjRgQ43TM'

# Construindo a rede

Para construir a rede é necessário criar um dicionário de de artistas com uma lista de artistas relacionados.

In [ ]:
def related_network(artist_id, depth: int=3) -> dict:
    """
    Retorna um dicionário de nomes de artistas às listas de seus artistas relacionados.
    
    Argumentos:
        arist_id: a id do artista de onde parte o grafo
        depth: profundidade da conexão na rede de artistas relacionados 
    
    Retorna: um dicionário de strings (nome do artista) para listas (artistas relacionados)
    """
    graph = dict()
    _related_network(artist_id, depth, graph)
    return graph

def _related_network(artist_id, depth, graph):
    """
    Coleta artistas relacionados e armazena os resultados no gráfico.
    
    Args:
        artist_id: id do artista do qual estão sendo extraídos os dados
        depth: o atual nível de profundidade do grafo
        graph: o dicionário para colocar os resultados do artista relacionado
    """
    if depth == 0:
        return
    name = sp.artist(artist_id)['name']
    if name in graph.keys(): 
      print("revisit for " + name)
      return
    print("fetching for " + name)
    like_artist = sp.artist_related_artists(artist_id)
    graph[name] = [related['name'] for related in like_artist['artists']]
    [_related_network(related['id'], depth - 1, graph) for related in like_artist['artists']]

In [ ]:
# Conferindo Milton Nascimento
related_network(seed_artist, 1)

fetching for Milton Nascimento


{'Milton Nascimento': ['Beto Guedes',
  'Lô Borges',
  'Gilberto Gil',
  'Tom Zé',
  'João Bosco',
  'Secos & Molhados',
  'Novos Baianos',
  'Luiz Melodia',
  'Jards Macalé',
  'Belchior',
  'Gal Costa',
  'Boca Livre',
  'Gonzaguinha',
  'Dorival Caymmi',
  'Ney Matogrosso',
  'Sérgio Sampaio',
  'Ivan Lins',
  'Caetano Veloso',
  'Nara Leão',
  'Itamar Assumpção']}

In [ ]:
like_seed = related_network(seed_artist, 5)

fetching for Milton Nascimento
fetching for Beto Guedes
fetching for Boca Livre
revisit for Beto Guedes
fetching for 14 Bis
fetching for Oswaldo Montenegro
fetching for Guilherme Arantes
revisit for Boca Livre
fetching for Renato Russo
revisit for Beto Guedes
fetching for Dalto
fetching for Flavio Venturini
fetching for Kleiton & Kledir
fetching for Marina Lima
fetching for Leo Jaime
fetching for Vander Lee
fetching for Ritchie
fetching for A Cor Do Som
fetching for Kiko Zambianchi
fetching for Os Paralamas Do Sucesso
fetching for Roupa Nova
fetching for Zélia Duncan
fetching for Fagner
fetching for Lobão
fetching for Pepeu Gomes
revisit for A Cor Do Som
fetching for Danilo Caymmi
fetching for Dori Caymmi
fetching for Nana Caymmi
fetching for Leila Pinheiro
fetching for Francis Hime
fetching for Johnny Alf
fetching for Cauby Peixoto
fetching for Miúcha
fetching for Ivan Lins
revisit for Boca Livre
fetching for João Bosco
fetching for Emílio Santiago
fetching for Leny Andrade
fetching f

# Formatando a rede para o Gephi

In [ ]:
import pandas as pd
import numpy as np

## nodesMiltonNascimento.csv
Nós

In [ ]:
def nodes(graph: dict) -> pd.DataFrame:
    """
    Retorna um dataframe de nós para o grafo.
    

    Graph: gera uma tabela única de nós
        
    Returns: um dataframe com nós e ids únicos
    """
    _nodes = []

    # itera sobre todos os artistas na lista
    for artist, related_list in graph.items():
        _nodes.append(artist)
        [_nodes.append(related) for related in related_list]

    # mantém apenas nós únicos
    _nodes = np.unique(_nodes)
    # faz um dataframe para gerar ids
    _nodes = pd.DataFrame(_nodes, columns=['label'])
    # usa as colunas de índice como o id
    _nodes['id'] = _nodes.index
    return _nodes

In [ ]:
network_nodes = nodes(like_seed)
network_nodes.head(10)

,label,id
0,14 Bis,0
1,3 Na Massa,1
2,A Bolha,2
3,A Cor Do Som,3
4,Abel Ferreira,4
5,Academia da Berlinda,5
6,Adoniran Barbosa,6
7,Adriana Calcanhotto,7
8,Adriana Partimpim,8
9,Agepê,9


## edgesMiltonNascimento.csv

Arestas

In [ ]:
def edges(graph: dict, nodes: pd.DataFrame) -> pd.DataFrame:
    """
    Retorna um dataframe de arestas com base no gráfico e na tabela de IDs de nós.
    
    Args:
        graph: o gráfico para encontrar arestas
        nodes: a tabela de nós com IDs de nós únicos
        
    """
    _edges = []

    for artist, related_list in graph.items():
        artist_node = nodes['id'][nodes['label'] == artist].values[0]
        for related in related_list:
            related_node = nodes['id'][nodes['label'] == related].values[0]
            _edges.append((artist_node, related_node))

    return pd.DataFrame(_edges, columns=['Source','Target'])

In [ ]:
network_edges = edges(like_seed, network_nodes)
network_edges.head()

,Source,Target
0,301,62
1,301,274
2,301,195
3,301,451
4,301,237


## Salvando os CSVs 

In [ ]:
from google.colab import files
network_nodes.to_csv('nodesMiltonNascimento.csv', index = False) 
files.download('nodesMiltonNascimento.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
network_edges.to_csv('edgesMiltonNascimento.csv', index = False) 
files.download('edgesMiltonNascimento.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>